In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.cluster import KMeans
import time 
import datetime
from datetime import date

In [2]:
%%time
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_train = pd.read_csv('BPI_Challenge_2012-training.csv')
df_test = pd.read_csv('BPI_Challenge_2012-test.csv')

df_data = pd.concat([df_train, df_test])

unixTransform = lambda x: time.mktime(x.timetuple())
secondsTransform = lambda x: x.total_seconds()

df_data["event time:timestamp"] = pd.to_datetime(df_data["event time:timestamp"], dayfirst=True)

Wall time: 25.3 s


In [3]:
df_data.sort_values(by=['event time:timestamp'], inplace=True)

In [4]:
def EventTime(data):
    
    for i in list(data["case concept:name"].unique()):
        data.loc[data["case concept:name"] == i, "nextTIME"] = data.loc[data["case concept:name"] == i, "event time:timestamp"].shift(-1)
    return data

In [5]:
%%time
# Get unique activities and encode them
Y = df_data['event concept:name'].unique()
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)

df_data = df_data.loc[df_data["event lifecycle:transition"]=="COMPLETE"]

df_encoded = df_data.loc[:, df_data.columns != 'event lifecycle:transition'].copy()

df_encoded['event concept:name'] = label_encoder.transform(df_data['event concept:name'])

df_encodedTime = EventTime(df_encoded)

df_encodedTime["Completion Time"] = np.nan
df_encodedTime["Completion Time"] = (df_encodedTime["nextTIME"] - df_encodedTime["event time:timestamp"]).apply(secondsTransform)
df_encodedTime.dropna(inplace=True)

df_encodedTime["Time Average"] = 0

for name, group in df_encodedTime[["event concept:name", "Completion Time"]].groupby(["event concept:name"]):
    mean = group["Completion Time"].mean()
    df_encodedTime.loc[df_encodedTime["event concept:name"] == name, "Time Average"] = mean

Wall time: 27 s


In [6]:
X = df_encodedTime[['event concept:name', "case AMOUNT_REQ", "Time Average"]]
y = df_encodedTime['Completion Time']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

y_pred = regr.predict(x_test)
# % of negative predicted values
(len(y_pred[y_pred<0])/len(y_pred))*100

16.10311275481002

In [7]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 67653.72200249211
Mean Squared Error: 22503656176.38821
Root Mean Squared Error: 150012.18675957035
